In [19]:
import pandas as pd
import tweepy
import numpy as np
import json
import datetime as datetime
import matplotlib.pyplot as plt
import seaborn as sns

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [20]:
twitter_consumer_key = "ThPe6egpT3OJCiRfx7w7lGyVH"
twitter_consumer_secret = "EnGwBptSnjSKEjgIbOi3D4qtRvfjcgnCWm3pDGQscjs07qXbG2"
twitter_access_token = "969396072215818240-ZVyRxct50k8giGAXzKoAVd4uThbuxSu"
twitter_access_token_secret = "tgyVOelmg2LN0FdlbrW37rh55DvMtWzwxj7oQtYRkBy81"

auth = tweepy.OAuthHandler(twitter_consumer_key, twitter_consumer_secret)
auth.set_access_token(twitter_access_token, twitter_access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [21]:
# Twitter accounts to read
news_user = ("@BBC","@CBSNews", "@CNN", "@FoxNews", "@NYTimes")

In [22]:
# setting default values
users = []
sentiment = []
neg = []
pos = []
neut = []
users = []
num = []
dates = []
text = []
oldest_tweet = None

counter = 0

In [23]:
for user in news_user:
   # 100 most recent tweets
    public_tweets = api.user_timeline(user, count=100, result_type="recent", max_id=oldest_tweet)
    
    counter = 1

    for tweet in public_tweets:
        # sentiment analysis on each tweet
        results = analyzer.polarity_scores(tweet["text"])

        # adding to arrays
        sentiment.append(results["compound"])
        neg.append(results["neg"])
        pos.append(results["pos"])
        neut.append(results["neu"])
        users.append(user)
        dates.append(tweet["created_at"])
        text.append(tweet["text"])
        num.append(counter)
        counter +=1
        
sentiment_df = pd.DataFrame({"Channel":users, 
                        "Date":dates,
                        "Compound":sentiment,
                        "Negative":neg, 
                        "Positive":pos,
                        "Neutral":neut, 
                        "Text":text,
                        "Tweet Ago": num})

In [24]:
sentiment_df.head()

,Channel,Compound,Date,Negative,Neutral,Positive,Text,Tweet Ago
0,@BBC,0.5515,Mon Mar 12 20:17:27 +0000 2018,0.000,0.771,0.229,"RT @BBCOne: ❤️ Edina, but kind of glad she's n...",1
1,@BBC,-0.7003,Mon Mar 12 20:16:22 +0000 2018,0.303,0.599,0.099,RT @BBCnireland: Christopher suffered a severe...,2
2,@BBC,0.2263,Mon Mar 12 19:23:01 +0000 2018,0.000,0.840,0.160,🌊 These three women are leading the way in oce...,3
3,@BBC,0.2263,Mon Mar 12 18:30:06 +0000 2018,0.000,0.881,0.119,🗓🥗 A week’s worth of veggie dishes from @ChefT...,4
4,@BBC,0.5859,Mon Mar 12 17:30:05 +0000 2018,0.000,0.840,0.160,'There's nothing in the dictionary for that.'\...,5


In [25]:
sentiment_df.to_csv("NewsMood.csv")

In [26]:
len(text)

500

In [33]:
plt.title("Sentiment Analysis of Media Tweets")
plt.ylabel("Tweet Polarity")
plt.xlabel("Tweets Ago")

news_labels = {'BBC': {'name': 'BBC', 'color': 'purple'},
               'CBSNews': {'name': 'CBS', 'color': 'red'},
               'CNN': {'name': 'CNN', 'color': 'blue'}, 
               'FoxNews': {'name': 'Fox', 'color': 'yellow'},
               'NYTimes': {'name': 'New York Times', 'color': 'green'}}

In [35]:
sns.lmplot(x="Tweet Ago", y="Compound", data=sentiment, fit_reg=False, hue="News Outlet", palette="Paired",legend=True,
           scatter_kws={"edgecolor":"grey", "linewidth":0.5, "alpha":0.8
           })

TypeError: list indices must be integers or slices, not list

In [ ]:
groupedScores=sentiment_csv.groupby("News Outlet")["Compound"].mean()
x_axis=np.arange(len(groupedScores))

sns.set()

plt.figure(figsize = (10,8))

for newsoutlet in news_labels.keys():
    df = sentiment_csv[sentiment_csv['News Outlet'] == newsoutlet]
    overall_media_sentiment = plt.bar(x_axis,groupedScores, color = {"purple","green", "yellow", "red", "blue"}, label = newsoutlet, edgecolor = "black")

plt.ylim(-.2, .3)
plt.ylabel("Tweet Polarity",fontweight='bold')
plt.axhline(y=0, color = 'black')

for i, v in enumerate(groupedScores):
    plt.text(i-.3, v+.03, str(v).format(), color='black', fontweight='bold')
    
plt.title("Overall Media Sentiment Based on Twitter (%s)" % (time.strftime("%m/%d/%Y")),fontweight='bold')
x_labels = ["BBC", "CBS", "CNN", "Fox", "NYT"]
x_locations = [value for value in np.arange(6)]
plt.xticks(x_locations, x_labels)
plt.show()
plt.savefig('Overall Sentiment Based on Twitter')